In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
from bayes_opt import UtilityFunction

from smac import HyperparameterOptimizationFacade, Scenario
from ConfigSpace import Categorical, Configuration, ConfigurationSpace, Float, Integer
from ConfigSpace import (
    ConfigurationSpace, UniformIntegerHyperparameter, UniformFloatHyperparameter, CategoricalHyperparameter
)

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

model_outputs = {}

# Comparing Different Bayesian Optimization Libraries and Grid Search

In this notebook, we will explore different libraries and methods for optimizing hyperparameters of machine learning models. We will compare grid search, Bayesian optimization with Gaussian Processes (GPs) using the `BayesianOptimization` library, Bayesian optimization with tree-structured Parzen estimators using the `Hyperopt` library, and Bayesian optimization with Random Forest regressions using the `SMAC3` library. These methods will be applied to optimize the hyperparameters of a Random Forest classifier from `sklearn` using the Iris dataset. This builds upon our previous understanding of Gaussian Processes and Bayesian optimization from the earlier notebooks.

## Baseline - Randomized Search with Cross-Validation

Grid search is a widely used technique for hyperparameter optimization. It involves an exhaustive search through a manually specified subset of the hyperparameter space. Although grid search can be computationally expensive and time-consuming, it is a simple and straightforward method for finding the best hyperparameters for a given model.

We will implement a grid search using the `GridSearchCV` function from `sklearn.model_selection`. This function allows us to perform an exhaustive search over a specified parameter grid, using cross-validation to assess the performance of each combination of hyperparameters.


In [2]:
# Define the hyperparameter search space
param_dist = {
    'n_estimators': np.arange(10, 150, dtype=int),
    'max_depth': np.arange(1, 20, dtype=int),
    'min_samples_split': np.random.uniform(0.1, 1, size=100),
    'min_samples_leaf': np.random.uniform(0.1, 0.5, size=100),
}

# Initialize the Random Forest classifier
clf = RandomForestClassifier()

# Run the randomized grid search
random_search = RandomizedSearchCV(
    clf, param_distributions=param_dist, n_iter=100, cv=5, n_jobs=-1
)
random_search.fit(X, y)

# Print the best hyperparameters
print("Best hyperparameters found:")
print("n_estimators:", random_search.best_params_['n_estimators'])
print("max_depth:", random_search.best_params_['max_depth'])
print("min_samples_split:", random_search.best_params_['min_samples_split'])
print("min_samples_leaf:", random_search.best_params_['min_samples_leaf'])

clf = RandomForestClassifier(**random_search.best_params_)
acc = cross_val_score(clf, X, y, cv=5).mean()
print("loss function:", -acc)

model_outputs['RandomizedSearchCV'] = {
    "n_estimators": random_search.best_params_['n_estimators'],
    "max_depth": random_search.best_params_['max_depth'],
    "min_samples_split": random_search.best_params_['min_samples_split'],
    "min_samples_leaf": random_search.best_params_['min_samples_leaf'],
    "loss": -acc
}

Best hyperparameters found:
n_estimators: 49
max_depth: 2
min_samples_split: 0.1637839854978797
min_samples_leaf: 0.19800723366913853
loss function: -0.9466666666666665


## Bayesian Optimisation with Gaussian Processes (Github: [BayesianOptimization](https://github.com/fmfn/BayesianOptimization))

The `BayesianOptimization` library is a Python package that provides a simple and efficient implementation of Bayesian optimization using Gaussian Processes. It allows us to optimize complex, expensive-to-evaluate functions with minimal function evaluations. The library is particularly useful for optimizing hyperparameters of machine learning models, as it can efficiently explore the hyperparameter space to find the best combination of hyperparameters.

We will use the `BayesianOptimization` library to optimize the hyperparameters of the Random Forest classifier.


In [3]:
# Define the hyperparameter search space
pbounds = {
    'n_estimators': (10, 150),
    'max_depth': (1, 20),
    'min_samples_split': (0.1, 1),
    'min_samples_leaf': (0.1, 0.5),
}

# Objective function
def objective(n_estimators, max_depth, min_samples_split, min_samples_leaf):
    params = {
        'n_estimators': int(n_estimators),
        'max_depth': int(max_depth),
        'min_samples_split': min_samples_split,
        'min_samples_leaf': min_samples_leaf,
    }
    clf = RandomForestClassifier(**params)
    acc = cross_val_score(clf, X, y, cv=5).mean()
    return acc

# Initialize the Bayesian Optimization with the kernel and acquisition function settings
optimizer = BayesianOptimization(
    f=objective,
    pbounds=pbounds,
    random_state=1,
    verbose=2,
) 

# Set the acquisition function (e.g., 'ucb', 'ei', or 'poi')
acquisition_function = 'ucb'
kappa = 2.576  # Exploration-exploitation parameter for UCB
xi = 0.0      # Exploration-exploitation parameter for EI and POI
utility = UtilityFunction(kind=acquisition_function, kappa=kappa, xi=xi)

# Run the Bayesian Optimization
for _ in range(100):
    next_point = optimizer.suggest(utility)
    target = objective(**next_point)
    optimizer.register(params=next_point, target=target)
    print(next_point, "- Target value = ", target)

# Print the best hyperparameters
print("Best hyperparameters found:")
print("n_estimators:", int(optimizer.max['params']['n_estimators']))
print("max_depth:", int(optimizer.max['params']['max_depth']))
print("min_samples_split:", optimizer.max['params']['min_samples_split'])
print("min_samples_leaf:", optimizer.max['params']['min_samples_leaf'])

model_outputs['BayesianOptimization'] = {
    "n_estimators": int(optimizer.max['params']['n_estimators']),
    "max_depth": int(optimizer.max['params']['max_depth']),
    "min_samples_split": optimizer.max['params']['min_samples_split'],
    "min_samples_leaf": optimizer.max['params']['min_samples_leaf'],
    "loss": -optimizer.max['target']
}

{'max_depth': 8.923418089348907, 'min_samples_leaf': 0.38812979737686326, 'min_samples_split': 0.1001029373356104, 'n_estimators': 52.32656016845757} - Target value =  0.3333333333333333
{'max_depth': 1.1466772636983, 'min_samples_leaf': 0.1647443736850785, 'min_samples_split': 0.7384321307529985, 'n_estimators': 117.06316025882413} - Target value =  0.3333333333333333
{'max_depth': 19.635105236299918, 'min_samples_leaf': 0.35628072635514696, 'min_samples_split': 0.8208045483256007, 'n_estimators': 10.103655336075324} - Target value =  0.3333333333333333
{'max_depth': 19.78814732966481, 'min_samples_leaf': 0.4527634743597063, 'min_samples_split': 0.12071819207446398, 'n_estimators': 149.96333229697035} - Target value =  0.3333333333333333
{'max_depth': 1.9098897597413482, 'min_samples_leaf': 0.40167210431205813, 'min_samples_split': 0.19577288031842527, 'n_estimators': 10.286754688103606} - Target value =  0.3333333333333333
{'max_depth': 1.0162404445650304, 'min_samples_leaf': 0.14738

## Advanced Bayesian Optimization with Gaussian Processes I (Github: [Trieste](https://github.com/secondmind-labs/trieste))

`Trieste` is an advanced Bayesian optimization library built on top of TensorFlow Probability. It offers a highly flexible and modular architecture, enabling users to easily customize various aspects of the optimization process. With its support for more sophisticated algorithms, `Trieste` is well-suited for tackling large-scale and high-dimensional optimization problems. In addition to traditional Gaussian Processes, `Trieste` allows users to employ more novel algorithms, some of which exhibit improved scalability properties, making it an attractive choice for optimizing complex functions.

Although we won't be relying on `Trieste` for this notebook, it is worth exploring if you require more advanced features or need to optimize hyperparameters in more complex machine learning models (see [tutorials](https://secondmind-labs.github.io/trieste/1.1.2/tutorials.html)). The library's flexibility and support for TensorFlow Probability make it a powerful tool for Bayesian optimization tasks that demand greater customization and scalability.

`Trieste` also provides an "[ask-tell](https://secondmind-labs.github.io/trieste/1.1.2/notebooks/ask_tell_optimization.html)" interface, which gives users greater control over the optimization loop. This interface is particularly useful in situations where allowing Trieste to manage the loop autonomously is not possible or desired. By separating the querying of new points (`ask`) from the reporting of function evaluations (`tell`), users can manually intervene in the optimization process, enabling more fine-grained control and customization.

The ask-tell interface is beneficial for scenarios where external factors influence the optimization process, such as hardware constraints or when the objective function requires human input. By employing the ask-tell interface, users can incorporate additional logic into the optimization loop, tailoring the algorithm to better suit their specific use case and requirements.

## Advanced Bayesian Optimization with Gaussian Processes II (Github: [BOtorch](https://github.com/pytorch/botorch))

[BOtorch](https://botorch.org/) is a flexible and efficient Bayesian optimization library built on top of PyTorch, GPyTorch, and Ax. It provides a modular and efficient implementation of Bayesian optimization algorithms, enabling researchers and practitioners to apply these methods to various real-world problems.

BOtorch is designed with the following goals in mind:

- **Modularity**: BOtorch allows users to build custom optimization algorithms by combining different components such as models, acquisition functions, and optimizers. This allows for rapid prototyping and experimentation.

- **Efficiency**: BOtorch leverages the power of PyTorch and GPyTorch for efficient tensor computations and automatic differentiation. This enables the library to handle large-scale problems with thousands of hyperparameters.

- **Interoperability**: BOtorch is compatible with other popular libraries, such as scikit-learn, and can be easily integrated into existing machine learning pipelines.

In this section, we will not explore how to use BOtorch to optimize the hyperparameters of the RandomForest classifier, as installation of torch can be tricky. Raw code below demonstrates nonetheless how to use Mixed GP models for handling mixed continuous and categorical hyperparameters, showcasing the flexibility and power of the BOtorch library for Bayesian optimization tasks.


## Bayesian Optimization with tree-structured Parzen Estimators (Github: [Hyperopt](https://github.com/hyperopt/hyperopt))

`Hyperopt` is another popular library for performing Bayesian optimization. Instead of using Gaussian Processes, `Hyperopt` employs tree-structured Parzen estimators (TPE) to model the objective function. TPE is a sequential model-based optimization approach that uses adaptive Parzen windows to approximate the true function.

We will use the `Hyperopt` library to optimize the hyperparameters of the Random Forest classifier (see [tutorial](https://github.com/hyperopt/hyperopt/wiki/FMin) for more details). A good blogpost on TPE is [this one](https://towardsdatascience.com/building-a-tree-structured-parzen-estimator-from-scratch-kind-of-20ed31770478).


In [4]:
import numpy as np
from functools import partial
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Define the objective function
def objective(params, X, y, cv):
    params = {
        'n_estimators': int(params['n_estimators']),
        'max_depth': int(params['max_depth']),
        'min_samples_split': params['min_samples_split'],
        'min_samples_leaf': params['min_samples_leaf'],
    }
    clf = RandomForestClassifier(**params)
    acc = cross_val_score(clf, X, y, cv=cv).mean()
    return {'loss': -acc, 'status': STATUS_OK}

# Define the search space
space = {
    'n_estimators': hp.quniform('n_estimators', 10, 150, 1),
    'max_depth': hp.quniform('max_depth', 1, 20, 1),
    'min_samples_split': hp.uniform('min_samples_split', 0.1, 1),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.1, 0.5),
} # If you have strong prior beliefs, sample around those beliefs with hp.normal instead
''' Example of a search space with strong prior beliefs
space = hp.choice('classifier_type', [
    {
        'type': 'naive_bayes',
    },
    {
        'type': 'svm',
        'C': hp.lognormal('svm_C', 0, 1),
        'kernel': hp.choice('svm_kernel', [
            {'ktype': 'linear'},
            {'ktype': 'RBF', 'width': hp.lognormal('svm_rbf_width', 0, 1)},
            ]),
    },
    {
        'type': 'dtree',
        'criterion': hp.choice('dtree_criterion', ['gini', 'entropy']),
        'max_depth': hp.choice('dtree_max_depth',
            [None, hp.qlognormal('dtree_max_depth_int', 3, 1, 1)]),
        'min_samples_split': hp.qlognormal('dtree_min_samples_split', 2, 1, 1),
    },
])
'''
# Configure the optimization
trials = Trials()
algo = tpe.suggest  # Tree-structured Parzen Estimator (TPE) algorithm
evals_per_optimization = 100

# Run the optimization
best = fmin(fn=partial(objective, X=X, y=y, cv=StratifiedKFold(5)),
            space=space,
            algo=algo,
            max_evals=evals_per_optimization,
            trials=trials,
            rstate=np.random.default_rng(1),
            verbose=2)

# Print the best hyperparameters
print("Best hyperparameters found:")
print("n_estimators:", int(best["n_estimators"]))
print("max_depth:", int(best["max_depth"]))
print("min_samples_split:", best["min_samples_split"])
print("min_samples_leaf:", best["min_samples_leaf"])

model_outputs['Hyperopt'] = {
    "n_estimators": int(best["n_estimators"]),
    "max_depth": int(best["max_depth"]),
    "min_samples_split": best["min_samples_split"],
    "min_samples_leaf": best["min_samples_leaf"],
    "loss": trials.best_trial['result']['loss']
}

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?][INFO][tpe.py:864] build_posterior_wrapper took 0.001367 seconds
[INFO][tpe.py:900] TPE using 0 trials
  1%|          | 1/100 [00:00<01:11,  1.39trial/s, best loss: -0.6866666666666665][INFO][tpe.py:864] build_posterior_wrapper took 0.001947 seconds
[INFO][tpe.py:900] TPE using 1/1 trials with best loss -0.686667
  2%|▏         | 2/100 [00:01<01:36,  1.02trial/s, best loss: -0.6866666666666665][INFO][tpe.py:864] build_posterior_wrapper took 0.002398 seconds
[INFO][tpe.py:900] TPE using 2/2 trials with best loss -0.686667
  3%|▎         | 3/100 [00:02<01:34,  1.03trial/s, best loss: -0.6866666666666665][INFO][tpe.py:864] build_posterior_wrapper took 0.001095 seconds
[INFO][tpe.py:900] TPE using 3/3 trials with best loss -0.686667
  4%|▍         | 4/100 [00:03<01:37,  1.02s/trial, best loss: -0.6866666666666665][INFO][tpe.py:864] build_posterior_wrapper took 0.002478 seconds
[INFO][tpe.py:900] TPE using 4/4 trials with best loss -0.6

## Bayesian Optimization with Random Forest regressions (Github: [SMAC3](https://github.com/automl/SMAC3))

`SMAC3` (Sequential Model-based Algorithm Configuration) is a versatile Bayesian optimization library that can use Random Forest regressions as surrogate models. `SMAC3` is particularly useful when dealing with high-dimensional, noisy, or expensive-to-evaluate objective functions. The library also provides features such as parallelization and support for categorical variables, making it a powerful tool for hyperparameter optimization.

We will use the `SMAC3` library to optimize the hyperparameters of the Random Forest classifier.

In [16]:
class RandomForest:
    # Define the objective function
    @property
    def configspace(self) -> ConfigurationSpace:
        """Note you can create dependencies between hyperparameters. Quite cool!
            class is `InCondition`, and you pass these onto the ConfigurationSpace via 
            `cs.add_conditions`

        Returns:
            ConfigurationSpace: A configuration space object.
        """        
        cs = ConfigurationSpace()
        n_estimators = UniformIntegerHyperparameter('n_estimators', 10, 150)
        max_depth = UniformIntegerHyperparameter('max_depth', 1, 20)
        min_samples_split = UniformFloatHyperparameter('min_samples_split', 0.1, 1)
        min_samples_leaf = UniformFloatHyperparameter('min_samples_leaf', 0.1, 0.5)
        criterion = CategoricalHyperparameter('criterion', ['gini', 'entropy'])
        cs.add_hyperparameters([n_estimators, max_depth, min_samples_split, min_samples_leaf, criterion])
        return cs
    
    def train(self, config: Configuration, seed: int = 0) -> float:
        config_dict = config.get_dictionary()
        clf = RandomForestClassifier(**config_dict, random_state=seed)
        acc = cross_val_score(clf, X, y).mean()
        return -acc
    
classifier = RandomForest()

# we create an object, holding general information about the run
scenario = Scenario(
    classifier.configspace,
    n_trials=50,  # We want to run max 50 trials (combination of config and seed)
)

# We want to run the facade's default initial design
initial_design = HyperparameterOptimizationFacade.get_initial_design(scenario, n_configs=5)

# Now we use SMAC to find the best hyperparameters
smac = HyperparameterOptimizationFacade(
    scenario,
    classifier.train,
    initial_design=initial_design,
    overwrite=True,  # If the run exists, we overwrite it; alternatively, we can continue from last state
)

# Run the optimization
incumbent = smac.optimize()

# Compute loss outside incumbent
inc_value = classifier.train(incumbent)

# Print the best hyperparameters
print("Best hyperparameters found:")
print("n_estimators:", int(incumbent["n_estimators"]))
print("max_depth:", int(incumbent["max_depth"]))
print("min_samples_split:", incumbent["min_samples_split"])
print("min_samples_leaf:", incumbent["min_samples_leaf"])

model_outputs['SMAC'] = {
    "n_estimators": int(incumbent["n_estimators"]),
    "max_depth": int(incumbent["max_depth"]),
    "min_samples_split": incumbent["min_samples_split"],
    "min_samples_leaf": incumbent["min_samples_leaf"],
    "loss": - classifier.train(incumbent)
}

[INFO][abstract_initial_design.py:69] Using `n_configs` and ignoring `n_configs_per_hyperparameter`.
[INFO][abstract_initial_design.py:134] Using 5 initial design configurations and 0 additional configurations.
[INFO][abstract_intensifier.py:513] Added config 3ed8ce as new incumbent because there are no incumbents yet.
[INFO][abstract_intensifier.py:588] Added config 4488bc and rejected config 3ed8ce as incumbent because it is not better than the incumbents on 3 instances:
[INFO][configspace.py:175] --- max_depth: 13 -> 4
[INFO][configspace.py:175] --- min_samples_leaf: 0.14300315268337727 -> 0.19232645876765278
[INFO][configspace.py:175] --- min_samples_split: 0.29059861330315473 -> 0.2872218170976268
[INFO][configspace.py:175] --- n_estimators: 131 -> 139
[INFO][abstract_intensifier.py:588] Added config 192079 and rejected config 4488bc as incumbent because it is not better than the incumbents on 3 instances:
[INFO][configspace.py:175] --- max_depth: 4 -> 19
[INFO][configspace.py:175

After running each of the four models, let's present a table comparing their relative performance in terms of the optimized hyperparameters and cross-validated accuracy. This comparison provides insights into the effectiveness of each method for hyperparameter optimization, and help us make an informed decision about which method to use in practice.

In [17]:
# Create table with model outputs
model_outputs = pd.DataFrame(model_outputs).T
model_outputs

,RandomizedSearchCV,BayesianOptimization,Hyperopt,SMAC
n_estimators,49.000000,149.000000,20.000000,57.000000
max_depth,2.000000,1.000000,5.000000,16.000000
min_samples_split,0.163784,0.502717,0.153527,0.126165
min_samples_leaf,0.198007,0.147383,0.207901,0.104829
loss,-0.946667,-0.960000,0.960000,NaN
SMAC,NaN,NaN,NaN,NaN


Things to bear in mind:
- Model overfitting: While Bayesian Optimization is less prone to overfitting compared to exhaustive search methods like grid search, it's still possible to overfit the model to the training data when searching for optimal hyperparameters. To prevent this, always use cross-validation during the optimization process. This provides a more reliable estimate of the model's performance on unseen data, which can help reduce the risk of overfitting.

- Complex surrogate model: Bayesian Optimization relies on a surrogate model (e.g., Gaussian Processes) to approximate the objective function. If this model is too complex, it may overfit the observed data, leading to overly optimistic predictions in certain regions of the search space. This could cause the optimization process to focus on regions that are not truly optimal. To reduce the risk of overfitting the surrogate model, it's important to choose an appropriate kernel function and its hyperparameters. In practice, a simpler kernel (e.g., a Radial Basis Function with fewer hyperparameters) is often preferred to avoid overfitting.

- Noise in the objective function: If the objective function is noisy, the surrogate model may struggle to fit the underlying trend, which could lead to suboptimal optimization. To address this, you can use techniques such as averaging multiple evaluations at the same point or using a more robust kernel that can handle noise better (e.g., the Matérn kernel).

- Number of iterations and exploration-exploitation trade-off: It is important to balance the number of iterations and the exploration-exploitation trade-off. Running too few iterations may lead to suboptimal results, while running too many iterations may result in over-exploitation of certain regions in the search space. Adjusting the exploration-exploitation parameters of the acquisition function (e.g., kappa for UCB and xi for EI and POI) can help balance this trade-off.